# 3.2 推理参数实验

本笔记本将通过实际实验,帮助你理解和掌握 LLM 的关键推理参数。

**预估成本**: ~$0.02 (使用 GPT-4o-mini)

**安装依赖**:
```bash
pip install openai
```

In [ ]:
# Install dependencies
!pip install openai -q

In [ ]:
import os
from openai import OpenAI
import json
from typing import List, Dict, Any

# Initialize客户端
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# 检查 API key
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  警告: 未设置 OPENAI_API_KEY 环境变量")
    print("请运行: export OPENAI_API_KEY='your-api-key'")
else:
    print("✓ API key 已设置")

In [ ]:
def call_llm(prompt: str, 
             model: str = "gpt-4o-mini",
             temperature: float = 1.0,
             top_p: float = 1.0,
             frequency_penalty: float = 0.0,
             presence_penalty: float = 0.0,
             max_tokens: int = 500) -> Dict[str, Any]:
    """调用 LLM API 并返回结果"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        max_tokens=max_tokens
    )
    
    return {
        "content": response.choices[0].message.content,
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        "model": response.model
    }

## Demo 1: Temperature ParametersExperiment
<!-- Demo 1: Temperature 参数实验 -->

**Temperature** 控制输出的随机性:
- **0**: 确定性输出,每次结果相同
- **0.5**: 中等随机性
- **1.0**: 默认值,平衡创造性和一致性
- **1.5+**: 高随机性,更有创意但可能不连贯

我们将用同一个提示词测试不同温度,每个温度运行 3 次观察变化。

In [ ]:
prompt = "用一句话描述人工智能的未来。"
temperatures = [0, 0.5, 1.0, 1.5]

print(f"提示词: {prompt}\n")
print("=" * 80)

for temp in temperatures:
    print(f"\n🌡️  Temperature = {temp}")
    print("-" * 80)
    
    responses = []
    for i in range(3):
        result = call_llm(prompt, temperature=temp, max_tokens=100)
        responses.append(result["content"])
        print(f"\n第 {i+1} 次: {result['content']}")
    
    # 检查是否所有响应都相同
    if len(set(responses)) == 1:
        print("\n📌 所有 3 次输出完全相同 (确定性)")
    else:
        unique_count = len(set(responses))
        print(f"\n📌 3 次输出中有 {unique_count} 个不同版本 (随机性)")

print("\n" + "=" * 80)
print("\n观察:")
print("- Temperature=0 时输出完全一致")
print("- Temperature 越高,输出越多样化")
print("- Temperature=1.5 时可能出现更有创意但风险更高的输出")

## Demo 2: Temperature vs Top-p Comparison
<!-- Demo 2: Temperature vs Top-p 对比 -->

**Top-p (nucleus sampling)** 是另一种控制随机性的方法:
- 只从累积概率达到 p 的 token 中采样
- top_p=0.1: 非常保守,只选择最可能的 tokens
- top_p=0.9: 平衡选择
- top_p=1.0: 考虑所有可能的 tokens

**注意**: OpenAI 建议只调整 temperature 或 top_p,不要同时调整。

In [ ]:
creative_prompt = "创作一个关于机器人学会做梦的故事开头。"

configs = [
    {"name": "Temperature=0.7", "temperature": 0.7, "top_p": 1.0},
    {"name": "Top-p=0.3 (保守)", "temperature": 1.0, "top_p": 0.3},
    {"name": "Top-p=0.9 (平衡)", "temperature": 1.0, "top_p": 0.9},
]

print(f"提示词: {creative_prompt}\n")
print("=" * 80)

for config in configs:
    print(f"\n⚙️  {config['name']}")
    print("-" * 80)
    
    result = call_llm(
        creative_prompt,
        temperature=config["temperature"],
        top_p=config["top_p"],
        max_tokens=150
    )
    
    print(f"\n{result['content']}")
    print(f"\n📊 Tokens: {result['usage']['completion_tokens']}")

print("\n" + "=" * 80)
print("\n观察:")
print("- Top-p=0.3 倾向于使用更常见、安全的词汇")
print("- Top-p=0.9 允许更多样化的词汇选择")
print("- 创意任务通常使用较高的 top_p 值")

## Demo 3: Frequency Penalty 和 Presence Penalty

这两个参数控制重复性:

**Frequency Penalty** (频率惩罚, -2.0 到 2.0):
- 根据 token 已经出现的次数进行惩罚
- 正值降低重复,负值增加重复

**Presence Penalty** (存在惩罚, -2.0 到 2.0):
- 根据 token 是否已经出现进行惩罚(不考虑次数)
- 正值鼓励谈论新话题,负值鼓励保持话题

让我们用一个容易产生重复的任务来测试。

In [ ]:
repetitive_prompt = "列举 10 个学习编程的好处,每个用一句话说明。"

penalty_configs = [
    {"name": "无惩罚 (默认)", "freq": 0.0, "pres": 0.0},
    {"name": "高频率惩罚", "freq": 1.5, "pres": 0.0},
    {"name": "高存在惩罚", "freq": 0.0, "pres": 1.5},
    {"name": "两者结合", "freq": 1.0, "pres": 1.0},
]

print(f"提示词: {repetitive_prompt}\n")
print("=" * 80)

for config in penalty_configs:
    print(f"\n⚙️  {config['name']}")
    print(f"   (frequency_penalty={config['freq']}, presence_penalty={config['pres']})")
    print("-" * 80)
    
    result = call_llm(
        repetitive_prompt,
        temperature=0.7,
        frequency_penalty=config["freq"],
        presence_penalty=config["pres"],
        max_tokens=300
    )
    
    print(f"\n{result['content']}")
    
    # 简单的重复检测
    words = result['content'].lower().split()
    unique_ratio = len(set(words)) / len(words) if words else 0
    print(f"\n📊 词汇多样性: {unique_ratio:.2%} ({len(set(words))}/{len(words)} 个独特词)")

print("\n" + "=" * 80)
print("\n观察:")
print("- Frequency penalty 减少常用词和短语的重复")
print("- Presence penalty 鼓励引入新话题和概念")
print("- 两者结合可以产生更多样化的输出")

## Demo 4: 不同任务的最佳Parameters
<!-- Demo 4: 不同任务的最佳参数 -->

根据任务类型,推荐不同的参数配置。

In [ ]:
# 定义不同任务场景和推荐参数
scenarios = [
    {
        "task": "数据提取",
        "prompt": '从以下文本提取人名、公司名和日期:\n"张三于2024年1月加入阿里巴巴公司,担任高级工程师。"',
        "params": {"temperature": 0, "top_p": 1.0, "frequency_penalty": 0, "presence_penalty": 0},
        "reason": "需要确定性输出,不需要创造性"
    },
    {
        "task": "创意写作",
        "prompt": "写一句赛博朋克风格的广告语,推广一款 AI 助手。",
        "params": {"temperature": 0.9, "top_p": 1.0, "frequency_penalty": 0.5, "presence_penalty": 0.5},
        "reason": "需要高创造性和词汇多样性"
    },
    {
        "task": "代码生成",
        "prompt": "用 Python 写一个函数,计算斐波那契数列的第 n 项。",
        "params": {"temperature": 0.2, "top_p": 1.0, "frequency_penalty": 0, "presence_penalty": 0},
        "reason": "需要精确性,但允许少量变化(如变量命名)"
    },
    {
        "task": "头脑风暴",
        "prompt": "为一个面向开发者的技术博客想 5 个创新栏目名称。",
        "params": {"temperature": 0.8, "top_p": 0.95, "frequency_penalty": 1.0, "presence_penalty": 0.6},
        "reason": "需要多样性和新颖性,避免重复想法"
    },
    {
        "task": "文本摘要",
        "prompt": "用一句话总结: LLM 通过在大规模数据上预训练学习语言模式,然后可以针对特定任务进行微调。它们使用 Transformer 架构,能够处理长距离依赖关系。",
        "params": {"temperature": 0.3, "top_p": 1.0, "frequency_penalty": 0, "presence_penalty": 0},
        "reason": "需要准确性和一致性,但允许一些改写"
    },
]

print("不同任务的推荐参数配置\n")
print("=" * 80)

for scenario in scenarios:
    print(f"\n📋 任务类型: {scenario['task']}")
    print(f"理由: {scenario['reason']}")
    print(f"\n推荐参数:")
    for key, value in scenario['params'].items():
        print(f"  {key}: {value}")
    
    print(f"\n提示词: {scenario['prompt']}")
    print("-" * 80)
    
    result = call_llm(
        scenario['prompt'],
        **scenario['params'],
        max_tokens=200
    )
    
    print(f"输出:\n{result['content']}")
    print("\n" + "=" * 80)

## Parameters速查表
<!-- 参数速查表 -->

根据实验结果,这里是一个实用的参数配置指南:

In [ ]:
import pandas as pd

# Create参数配置表
config_table = pd.DataFrame([
    {
        "任务类型": "数据提取/分类",
        "Temperature": "0",
        "Top-p": "1.0",
        "Freq Penalty": "0",
        "Pres Penalty": "0",
        "说明": "需要一致性和准确性"
    },
    {
        "任务类型": "代码生成",
        "Temperature": "0-0.2",
        "Top-p": "1.0",
        "Freq Penalty": "0",
        "Pres Penalty": "0",
        "说明": "精确但允许命名变化"
    },
    {
        "任务类型": "文本摘要",
        "Temperature": "0.3",
        "Top-p": "1.0",
        "Freq Penalty": "0",
        "Pres Penalty": "0",
        "说明": "准确但允许改写"
    },
    {
        "任务类型": "问答/对话",
        "Temperature": "0.7",
        "Top-p": "0.9",
        "Freq Penalty": "0.3",
        "Pres Penalty": "0.3",
        "说明": "平衡准确性和自然度"
    },
    {
        "任务类型": "创意写作",
        "Temperature": "0.8-1.0",
        "Top-p": "1.0",
        "Freq Penalty": "0.5-1.0",
        "Pres Penalty": "0.5-1.0",
        "说明": "鼓励创造性和多样性"
    },
    {
        "任务类型": "头脑风暴",
        "Temperature": "0.9",
        "Top-p": "0.95",
        "Freq Penalty": "1.0-1.5",
        "Pres Penalty": "0.6-1.0",
        "说明": "最大化多样性和新颖性"
    },
    {
        "任务类型": "长文本生成",
        "Temperature": "0.7",
        "Top-p": "0.9",
        "Freq Penalty": "0.5",
        "Pres Penalty": "0.3",
        "说明": "避免重复同时保持连贯"
    },
])

print("\n推理参数配置速查表\n")
print(config_table.to_string(index=False))

print("\n\n💡 重要提示:")
print("1. OpenAI 建议只调整 temperature 或 top_p,不要同时调整")
print("2. 从推荐值开始,根据实际效果微调")
print("3. Temperature=0 对于需要确定性的任务至关重要")
print("4. Penalty 参数对长文本和多轮对话效果更明显")
print("5. 创意任务可以大胆尝试更高的参数值")

## 练习: 为技术Document摘要找到最佳 Temperature
<!-- 练习: 为技术文档摘要找到最佳 Temperature -->

尝试不同的 temperature 值,找出最适合摘要任务的设置。

In [ ]:
# 技术文档示例
tech_doc = """
React 18 引入了并发渲染(Concurrent Rendering)特性,这是 React 架构的重大升级。
并发渲染允许 React 同时准备多个版本的 UI,可以中断渲染过程来处理更高优先级的更新。
新增的 useTransition 和 useDeferredValue Hooks 让开发者能够标记非紧急更新,
从而在不阻塞用户交互的情况下更新 UI。此外,Suspense 组件现在支持服务端渲染,
可以在数据加载时显示 fallback UI,改善用户体验。自动批处理(Automatic Batching)
功能扩展到了 Promise、setTimeout 和原生事件处理器,减少了不必要的重渲染。
"""

summarize_prompt = f"用 2-3 句话总结以下技术文档的核心内容:\n\n{tech_doc}"

# Test不同的 temperature
test_temperatures = [0, 0.3, 0.5, 0.7]

print("练习: 为技术文档摘要找到最佳 Temperature\n")
print("=" * 80)
print(f"原文 ({len(tech_doc)} 字符):\n{tech_doc}")
print("\n" + "=" * 80)

summaries = {}
for temp in test_temperatures:
    print(f"\n🌡️  Temperature = {temp}")
    print("-" * 80)
    
    result = call_llm(summarize_prompt, temperature=temp, max_tokens=200)
    summaries[temp] = result['content']
    
    print(f"{result['content']}")
    print(f"\n📊 字数: {len(result['content'])} | Tokens: {result['usage']['completion_tokens']}")

print("\n" + "=" * 80)
print("\n分析与评估:")
print("\n请回答以下问题:")
print("1. 哪个 temperature 产生的摘要最准确?")
print("2. 哪个 temperature 产生的摘要最简洁?")
print("3. 不同 temperature 的输出差异大吗?")
print("4. 对于技术文档摘要,你会选择哪个 temperature?为什么?")
print("\n💡 提示: 对于事实性任务,较低的 temperature (0-0.3) 通常更好")

## 总结

通过本笔记本的实验,你应该掌握了:

### 1. 核心参数

**Temperature (0-2)**:
- 控制输出的随机性和创造性
- 0 = 确定性,适合需要精确答案的任务
- 0.7-1.0 = 平衡,适合大多数任务
- 1.0+ = 高创造性,适合创意写作

**Top-p (0-1)**:
- 替代 temperature 的采样方法
- 建议不要与 temperature 同时调整
- 0.1 = 保守,0.9 = 平衡,1.0 = 全部考虑

**Frequency Penalty (-2 到 2)**:
- 根据出现次数惩罚 token
- 正值减少重复,适合长文本和列表

**Presence Penalty (-2 到 2)**:
- 根据是否出现过惩罚 token
- 正值鼓励新话题,适合头脑风暴和对话

### 2. 任务匹配

- **精确任务** (提取、分类、代码): Temperature = 0-0.2
- **平衡任务** (摘要、问答): Temperature = 0.3-0.7
- **创意任务** (写作、头脑风暴): Temperature = 0.8-1.0+

### 3. 最佳实践

1. 从保守参数开始,逐步增加随机性
2. 对关键任务使用 temperature=0 确保一致性
3. 使用 penalty 参数减少长文本中的重复
4. 多次测试找到最适合你任务的参数
5. 记录成功的参数配置以便复用